# Simulating Prisoner's Dilemma

In this game we're simulating the interactions between different strategies in
the Prisoner's Dilemma.  
Each strategy is a function that takes the history of the opposing strategy and
own history and returns a decision (cooperate or defect).

The game makes no assumptions about the internals or state of the strategies.
They may keep state over multiple turns or be completely stateless.  
For simplicity, it is recommended to depend entirely on the values passed in
each turn.  

Strategies that depend on external information, to this game, would be
considered cheaters.  

Strategies can see each other's scores as we want to allow behaviour that
minimises other strategies score, as opposed to maximising self.

### Setup

In [60]:
from typing import Callable, Any, Iterable
from collections import defaultdict

## The Game Loop

The game loop is stable and shouldn't change.  
We may implement different games in the future. One example would be to simulate
a NxN game, where multiple strategies go against multiple strategies.

In [36]:
Move = str
Score = int
Turn = tuple[Move, Score]
History = list[Turn]
Strategy = Callable[[History, History], Move]

ScoreFunc = Callable[[Move, Move], tuple[Score, Score]]

In [37]:
def simulate_game(strategy1: Strategy, strategy2: Strategy,
                  update_scores: ScoreFunc, num_turns: int
) -> tuple[Score, Score]:
    """Simulate a prisoner's dilemma game and return the total scores for
    both players.
    Each strategy takes the history of the opposing player and self history and
    returns the next move.
    
    The game returns the total scores for both players."""
    history1, history2 = [], []
    total_score1, total_score2 = 0, 0

    for _ in range(num_turns):
        move1 = strategy1(history2, history1)
        move2 = strategy2(history1, history2)

        score1, score2 = update_scores(move1, move2)
        total_score1 += score1
        total_score2 += score2

        history1.append((move1, score1))
        history2.append((move2, score2))
    
    return total_score1, total_score2

In [16]:
COOPERATE = "C"
DEFECT = "D"

### The Rules

Let's define our scoring functions.

In [19]:
ScoreMatrix = dict[tuple[Move, Move], tuple[Score, Score]]

In [20]:
def create_score_func_from_matrix(score_matrix: ScoreMatrix) -> ScoreFunc:
    
    def update_scores(move1, move2) -> tuple[Score, Score]:
        return score_matrix[(move1, move2)]
    
    return update_scores

In [22]:
classic_score_matrix: ScoreMatrix = {
    (COOPERATE, COOPERATE): (3, 3),
    (COOPERATE, DEFECT): (0, 5),
    (DEFECT, COOPERATE): (5, 0),
    (DEFECT, DEFECT): (1, 1)
}
update_classic_scores = create_score_func_from_matrix(classic_score_matrix)

In [23]:
# Let's test it
update_classic_scores(COOPERATE, COOPERATE)

(3, 3)

## Let's Play

Let's define some strategies and run the game

### Strategy Library

#### Optimist
Simple strategy, always cooperates.

In [40]:
def optimist(_opposing: History, _own: History) -> Move:
    return COOPERATE

#### Criminal
Always defects

In [41]:
def criminal(_opposing: History, _own) -> Move:
    return DEFECT

#### Tit for Tat
This strategy starts optimistic and then repeats what the opposing strategy's previous turn from then on.

In [42]:
def tit_for_tat(opposing: History, _own: History) -> Move:
    if not opposing:
        return COOPERATE
    
    previous_move, _ = opposing[-1]
    return previous_move

### Games

TODO:
- Change the code to take a list of strategies to evaluate and render the leaderboard comparing each strategy

In [43]:
# Let's test the simulation works
simulate_game(criminal, criminal, update_classic_scores, 200)

(200, 200)

In [48]:
simulate_game(criminal, tit_for_tat, update_classic_scores, 100)

(104, 99)

## Evaluating Strategies

Let's evaluate the different strategies available.  
We do so by looking at a few metrics:  
* Total score across all games  
* Best game score  
* Worst game score  
* Number of wins  
* Number of ties  

To calculate these metrics we run the combination of all strategies, 
including playing against self.  
We end up with a results matrix representing the scores between strategy A and B
where the diagonal represents playing against self.

We then plot the results of processing this matrix into the various metrics.

In [44]:
strategies_to_compare = [
    optimist, criminal, tit_for_tat
]

In [51]:
# TODO: Replace this with the python combinatorial function
def combinations(items: list[Any]) -> Iterable[tuple[Any, Any]]:
    for i in items:
        for j in items:
            yield (i, j)

In [59]:
NUM_TURNS = 200

def to_key(strategy1: Strategy, strategy2: Strategy) -> tuple[str, str]:
    return (strategy1.__name__, strategy2.__name__)

def simulate_classic_game(
    strategy1: Strategy, strategy2: Strategy
) -> tuple[Score, Score]:
    """Curries simulate_game to use classic scores"""
    return simulate_game(strategy1, strategy2, update_classic_scores, NUM_TURNS)

results = {to_key(strategy1, strategy2): simulate_classic_game(strategy1, strategy2)
           for strategy1, strategy2 in combinations(strategies_to_compare)}
results

{('optimist', 'optimist'): (600, 600),
 ('optimist', 'criminal'): (0, 1000),
 ('optimist', 'tit_for_tat'): (600, 600),
 ('criminal', 'optimist'): (1000, 0),
 ('criminal', 'criminal'): (200, 200),
 ('criminal', 'tit_for_tat'): (204, 199),
 ('tit_for_tat', 'optimist'): (600, 600),
 ('tit_for_tat', 'criminal'): (199, 204),
 ('tit_for_tat', 'tit_for_tat'): (600, 600)}

In [65]:
totals = defaultdict(int)
best = defaultdict(int)
wins = defaultdict(int)
ties = defaultdict(int)

# TODO: Improve with a flatter results list
for (strategy1, strategy2), (score1, score2) in results.items():
    totals[strategy1] += score1
    totals[strategy2] += score2

    best[strategy1] = max(best[strategy1], score1)
    best[strategy2] = max(best[strategy2], score2)

    if score1 > score2:
        wins[strategy1] += 1
    elif score1 == score2:
        ties[strategy1] += 1
        ties[strategy2] += 1
    else:
        wins[strategy2] += 1

dict(totals), dict(best), dict(wins), dict(ties)

({'optimist': 2400, 'criminal': 2808, 'tit_for_tat': 2798},
 {'optimist': 600, 'criminal': 1000, 'tit_for_tat': 600},
 {'criminal': 4},
 {'optimist': 4, 'tit_for_tat': 4, 'criminal': 2})